In [41]:
import polars as pl
import altair as alt

research_ethics_score = pl.read_csv("data/5_1_research_ethics_score.csv")

In [42]:
def draw_bar_plot(df, x, y, title, MAP, rotate=True, legend=True, scale=None):

    bar_x = alt.X(x, title=title, axis=None)
    bar_x = bar_x if MAP is None else bar_x.sort(MAP.values())

    bar_y = alt.Y(y, title=None, scale=scale)

    lgd_x = alt.Legend(orient='bottom', direction='horizontal')
    color = alt.Color(x, title=None, legend=lgd_x) if legend else alt.Color(x, title=None, legend=None)
    color = color.sort(MAP.values()) if MAP is not None else color

    bar = df.plot.bar().encode(
        x=bar_x,
        y=bar_y,
        color=color,
    ).properties(
        title=title,
        width=400,
    )

    txt_x = alt.X(x, sort=MAP.values()) if MAP is not None else alt.X(x)
    text = bar.mark_text(
        align="center",
        baseline="bottom",
    ).encode(
        x=txt_x,
        y=y,
        text=alt.Text(y, format=".2f"),
        color=alt.value("black"),
    )
    return bar + text

In [43]:
#
# 가. 문항별 기술 통계량
#
mean = research_ethics_score.mean().drop("ID").write_csv("figure/5/가_문항별_기술_통계량_평균.csv", include_bom=True)
standard_deviaiton = research_ethics_score.std().drop("ID").write_csv("figure/5/가_문항별_기술_통계량_표준편차.csv", include_bom=True)

In [44]:
# 
# 나. 영역별 기술 통계량
# 
# *Q39_1: 동물실험에 대한 연구윤리
#
공정 = (pl.col("Q38_1") + pl.col("Q38_2") + pl.col("Q38_3") + pl.col("Q38_4")) / 4
정직 = (pl.col("Q38_5") + pl.col("Q38_6") + pl.col("Q38_7") + pl.col("Q38_8")) / 4
존중 = pl.when(pl.col("Q39_1").is_null()).then(
    (pl.col("Q38_9") + pl.col("Q38_10") + pl.col("Q38_11")) / 3
).otherwise(
    (pl.col("Q38_9") + pl.col("Q38_10") + pl.col("Q38_11") + pl.col("Q39_1")) / 4
)

책임 = (pl.col("Q38_12") + pl.col("Q38_13") + pl.col("Q38_14") + pl.col("Q38_15")) / 4
투명성 = (pl.col("Q38_16") + pl.col("Q38_17") + pl.col("Q38_18") + pl.col("Q38_19")) / 4
all = (공정 + 정직 + 존중 + 책임 + 투명성) / 5

by_area = research_ethics_score.select([
    pl.col("ID"),
    공정.alias("공정"),
    정직.alias("정직"),
    존중.alias("존중"),
    책임.alias("책임"),
    투명성.alias("투명성"),
    all.alias("전체")
])

by_area_mean = by_area.mean()
by_area_mean.write_csv("figure/5/나_영역별_기술_통계량_평균.csv", include_bom=True)
by_area_std = by_area.std()
by_area_std.write_csv("figure/5/나_영역별_기술_통계량_표준편차.csv", include_bom=True)


In [45]:
tp_by_area = by_area_mean.drop("ID").transpose(include_header=True)

bar = draw_bar_plot(tp_by_area, "column", "column_0", "영역별 기술 통계량", None, rotate=False, legend=True, scale=alt.Scale(domain=[4, 4.8]))
bar.show()

alt.LayerChart(...)

In [46]:
# 
# 다. 판정결과
# 

def make_grade(column, excellent, great, good, bad):
    return pl.when(column >= excellent).then(5).when(column >= great).then(4).when(column >= good).then(3).when(column >= bad).then(2).otherwise(1)

grading_all = make_grade(pl.col("전체"), 4.70, 3.85, 3.40, 2.95)
grading_공정 = make_grade(pl.col("공정"), 5.0, 3.75, 3.0, 2.50)
grading_정직 = make_grade(pl.col("정직"), 5.0, 4.0, 3.5, 3.25)
grading_존중 = make_grade(pl.col("존중"), 5.0, 3.75, 3.0, 2.25)
grading_책임 = make_grade(pl.col("책임"), 5.0, 3.75, 3.25, 2.75)
grading_투명성 = make_grade(pl.col("투명성"), 5.0, 3.75, 3.25, 2.75)

grading = by_area.select([
    grading_all.alias("전체_채점"),
    grading_공정.alias("공정_채점"),
    grading_정직.alias("정직_채점"),
    grading_존중.alias("존중_채점"),
    grading_책임.alias("책임_채점"),
    grading_투명성.alias("투명성_채점"),
])

print(grading)

shape: (167, 6)
┌───────────┬───────────┬───────────┬───────────┬───────────┬─────────────┐
│ 전체_채점 ┆ 공정_채점 ┆ 정직_채점 ┆ 존중_채점 ┆ 책임_채점 ┆ 투명성_채점 │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---         │
│ i32       ┆ i32       ┆ i32       ┆ i32       ┆ i32       ┆ i32         │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══════════╪═════════════╡
│ 3         ┆ 4         ┆ 3         ┆ 3         ┆ 3         ┆ 2           │
│ 4         ┆ 4         ┆ 4         ┆ 4         ┆ 4         ┆ 4           │
│ 4         ┆ 4         ┆ 4         ┆ 4         ┆ 4         ┆ 4           │
│ 5         ┆ 4         ┆ 5         ┆ 5         ┆ 4         ┆ 5           │
│ 4         ┆ 4         ┆ 4         ┆ 4         ┆ 4         ┆ 4           │
│ …         ┆ …         ┆ …         ┆ …         ┆ …         ┆ …           │
│ 3         ┆ 4         ┆ 3         ┆ 3         ┆ 3         ┆ 4           │
│ 4         ┆ 4         ┆ 3         ┆ 4         ┆ 4         ┆ 4           │
│ 2         ┆ 3      

In [47]:
all_count = grading.group_by("전체_채점").agg(pl.count("전체_채점").alias("전체")).rename({"전체_채점": "등급"})
공정_count = grading.group_by("공정_채점").agg(pl.count("공정_채점").alias("공정")).rename({"공정_채점": "등급"})
정직_count = grading.group_by("정직_채점").agg(pl.count("정직_채점").alias("정직")).rename({"정직_채점": "등급"})
존중_count = grading.group_by("존중_채점").agg(pl.count("존중_채점").alias("존중")).rename({"존중_채점": "등급"})
책임_count = grading.group_by("책임_채점").agg(pl.count("책임_채점").alias("책임")).rename({"책임_채점": "등급"})
투명성_count = grading.group_by("투명성_채점").agg(pl.count("투명성_채점").alias("투명성")).rename({"투명성_채점": "등급"})

def check_zero_and_concat(dataframe, column_name, basis=5):
    if dataframe.shape[0] < basis:
        t = pl.DataFrame({"count": [1], column_name: [0]})
        t = t.with_columns(
            pl.col("count").cast(pl.Int32),
            pl.col(column_name).cast(pl.UInt32),
        )
        return dataframe.extend(t)
    return dataframe

all_count = all_count.join(공정_count, on="등급", how="left") \
    .join(정직_count, on="등급", how="left") \
    .join(존중_count, on="등급", how="left") \
    .join(책임_count, on="등급", how="left") \
    .join(투명성_count, on="등급", how="left") \
    .fill_null(0) \
    .sort("등급")

all_count.write_csv("figure/5/다_연구윤리_영역별_연구윤리_수준_판정_결과_총합.csv", include_bom=True)

all_count.drop("등급").transpose(include_header=True).select([
    pl.col("column_0") / (pl.col("column_0") + pl.col("column_1") + pl.col("column_2") + pl.col("column_3") + pl.col("column_4")).alias("매우 미흡"),
    pl.col("column_1") / (pl.col("column_0") + pl.col("column_1") + pl.col("column_2") + pl.col("column_3") + pl.col("column_4")).alias("미흡"),
    pl.col("column_2") / (pl.col("column_0") + pl.col("column_1") + pl.col("column_2") + pl.col("column_3") + pl.col("column_4")).alias("보통"),
    pl.col("column_3") / (pl.col("column_0") + pl.col("column_1") + pl.col("column_2") + pl.col("column_3") + pl.col("column_4")).alias("우수"),
    pl.col("column_4") / (pl.col("column_0") + pl.col("column_1") + pl.col("column_2") + pl.col("column_3") + pl.col("column_4")).alias("매우 우수"),
]).write_csv("figure/5/다_연구윤리_영역별_연구윤리_수준_판정_결과_비율.csv", include_bom=True)

In [48]:
# 
# 라. 조사 대상별 차이 검정
# 
from scipy import stats
personal_info = pl.read_csv("data/6_personal_info.csv")

def group_per_personal(column):
    personality = personal_info.select([pl.col("ID"), pl.col(column)])
    by_area_per_personality = by_area.join(personality, on="ID", how="inner")
    return by_area_per_personality

def compute_group_personality_by_callback(by_area_per_personality, column, callback):
    return by_area_per_personality.group_by([column]).agg( \
        callback("공정"), \
        callback("정직"), \
        callback("존중"), \
        callback("책임"), \
        callback("투명성"), \
        callback("전체")
    ).sort(column).drop(column).transpose(include_header=True)

# target
components = ["공정", "정직", "존중", "책임", "투명성", "전체"]
pl_components = pl.col(components)


# 1) 성별
sex = "Q42"
by_area_per_sex = group_per_personal(sex)

count_per_sex = compute_group_personality_by_callback(by_area_per_sex, sex, pl.count).rename({"column_0": "남자수", "column_1": "여자수"})
mean_per_sex = compute_group_personality_by_callback(by_area_per_sex, sex, pl.mean).rename({"column_0": "남자평균", "column_1": "여자평균"})
std_per_sex = compute_group_personality_by_callback(by_area_per_sex, sex, pl.std).rename({"column_0": "남자표준편차", "column_1": "여자펴준편차"})

all = count_per_sex.join(
    mean_per_sex, on="column", how="inner", 
).join(
    std_per_sex, on="column", how="inner"
)
all.write_csv("figure/5/라_1_성별_조사_대상별_차이_검정_통계량.csv", include_bom=True)

male = by_area_per_sex.filter(pl.col(sex) == 1).select(pl_components)
female = by_area_per_sex.filter(pl.col(sex) == 2).select(pl_components)

ttest = stats.ttest_ind(male, female)
ttest_sex = pl.DataFrame({
    "component": components,
    "t-value": ttest[0],
    "p-value": ttest[1],
}).write_csv("figure/5/라_1_성별_조사_대상별_차이_검정_ttest.csv", include_bom=True)

In [49]:
# 2) 연령
age = "Q43"
by_area_per_age = group_per_personal(age)

count_per_age = compute_group_personality_by_callback(by_area_per_age, age, pl.count).rename({"column_0": "20대수", "column_1": "30대수", "column_2": "40대수", "column_3": "50대수", "column_4": "60대수"})
mean_per_age = compute_group_personality_by_callback(by_area_per_age, age, pl.mean).rename({"column_0": "20대평균", "column_1": "30대평균", "column_2": "40대평균", "column_3": "50대평균", "column_4": "60대평균"})
std_per_age = compute_group_personality_by_callback(by_area_per_age, age, pl.std).rename({"column_0": "20대표준편차", "column_1": "30대표준편차", "column_2": "40대표준편차", "column_3": "50대표준편차", "column_4": "60대표준편차"})

all = count_per_age.join(
    mean_per_age, on="column", how="inner", 
).join(
    std_per_age, on="column", how="inner"
)
all.write_csv("figure/5/라_2_연령대_조사_대상별_차이_검정_통계량.csv", include_bom=True)

age_20 = by_area_per_age.filter(pl.col(age) == 1).select(pl_components)
age_30 = by_area_per_age.filter(pl.col(age) == 2).select(pl_components)
age_40 = by_area_per_age.filter(pl.col(age) == 3).select(pl_components)
age_50 = by_area_per_age.filter(pl.col(age) == 4).select(pl_components)
age_60 = by_area_per_age.filter(pl.col(age) == 5).select(pl_components)

anova = stats.f_oneway(age_20, age_30, age_40, age_50, age_60)
anova_age = pl.DataFrame({
    "component": components,
    "f-value": anova[0],
    "p-value": anova[1],
}).write_csv("figure/5/라_2_연령대_조사_대상별_차이_검정_anova.csv", include_bom=True)

In [50]:
#
# 3) 직위
#
role = "Q44"
by_area_per_role = group_per_personal(role)

count_per_role = compute_group_personality_by_callback(by_area_per_role, role, pl.count).rename({"column_0": "책임급수", "column_1": "선임급수", "column_2": "원급수", "column_3": "무기(선임급)수", "column_4": "무기(원급)수", "column_5": "기타수"})
mean_per_role = compute_group_personality_by_callback(by_area_per_role, role, pl.mean).rename({"column_0": "책임급평균", "column_1": "선임급평균", "column_2": "원급평균", "column_3": "무기(선임급)평균", "column_4": "무기(원급)평균", "column_5": "기타평균"})
std_per_role = compute_group_personality_by_callback(by_area_per_role, role, pl.std).rename({"column_0": "책임급표준편차", "column_1": "선임급표준편차", "column_2": "원급표준편차", "column_3": "무기(선임급)표준편차", "column_4": "무기(원급)표준편차", "column_5": "기타표준편차"})

all = count_per_role.join(
    mean_per_role, on="column", how="inner", 
).join(
    std_per_role, on="column", how="inner"
)

all.write_csv("figure/5/라_3_직위_조사_대상별_차이_검정_통계량.csv", include_bom=True)

researcher = by_area_per_role.filter(pl.col(role) == 1).select(pl_components)
professor = by_area_per_role.filter(pl.col(role) == 2).select(pl_components)
manager = by_area_per_role.filter(pl.col(role) == 3).select(pl_components)
director = by_area_per_role.filter(pl.col(role) == 4).select(pl_components)
executive = by_area_per_role.filter(pl.col(role) == 5).select(pl_components)

anova = stats.f_oneway(researcher, professor, manager, director, executive)
anova_role = pl.DataFrame({
    "component": components,
    "f-value": anova[0],
    "p-value": anova[1],
}).write_csv("figure/5/라_3_직위_조사_대상별_차이_검정_anova.csv", include_bom=True)

In [51]:
# 
# 4) 학위
# 
degree = "Q45"
by_area_per_degree = group_per_personal(degree)

count_per_degree = compute_group_personality_by_callback(by_area_per_degree, degree, pl.count).rename({"column_0": "박사수", "column_1": "석사수", "column_2": "학사수"})
mean_per_degree = compute_group_personality_by_callback(by_area_per_degree, degree, pl.mean).rename({"column_0": "박사평균", "column_1": "석사평균", "column_2": "학사평균"})
std_per_degree = compute_group_personality_by_callback(by_area_per_degree, degree, pl.std).rename({"column_0": "박사표준편차", "column_1": "석사표준편차", "column_2": "학사표준편차"})

all = count_per_degree.join(
    mean_per_degree, on="column", how="inner", 
).join(
    std_per_degree, on="column", how="inner"
)

all.write_csv("figure/5/라_4_학위_조사_대상별_차이_검정_통계량.csv", include_bom=True)

bachelor = by_area_per_degree.filter(pl.col(degree) == 1).select(pl_components)
master = by_area_per_degree.filter(pl.col(degree) == 2).select(pl_components)
doctor = by_area_per_degree.filter(pl.col(degree) == 3).select(pl_components)
# postdoctor = by_area_per_degree.filter(pl.col(degree) == 4).select(pl_components)

anova = stats.f_oneway(bachelor, master, doctor)
anova_degree = pl.DataFrame({
    "component": components,
    "f-value": anova[0],
    "p-value": anova[1],
}).write_csv("figure/5/라_4_학위_조사_대상별_차이_검정_anova.csv", include_bom=True)

In [52]:
# 
# 5) 연구경력
# 

research_experience = "Q46"
by_area_per_research_experience = group_per_personal(research_experience)

count_per_research_experience = compute_group_personality_by_callback(by_area_per_research_experience, research_experience, pl.count).rename({"column_0": "5년미만", "column_1": "5년이상10년미만수", "column_2": "10년이상15년미만수", "column_3": "15년이상수"})
mean_per_research_experience = compute_group_personality_by_callback(by_area_per_research_experience, research_experience, pl.mean).rename({"column_0": "5년미만평균", "column_1": "5년이상10년미만평균", "column_2": "10년이상15년미만평균", "column_3": "15년이상평균"})
std_per_research_experience = compute_group_personality_by_callback(by_area_per_research_experience, research_experience, pl.std).rename({"column_0": "5년미만표준편차", "column_1": "5년이상10년미만표준편차", "column_2": "10년이상15년미만표준편차", "column_3": "15년이상표준편차"})

all = count_per_research_experience.join(
    mean_per_research_experience, on="column", how="inner", 
).join(
    std_per_research_experience, on="column", how="inner"
)

all.write_csv("figure/5/라_5_연구경력_조사_대상별_차이_검정_통계량.csv", include_bom=True)

experience_1 = by_area_per_research_experience.filter(pl.col(research_experience) == 1).select(pl_components)
experience_2 = by_area_per_research_experience.filter(pl.col(research_experience) == 2).select(pl_components)
experience_3 = by_area_per_research_experience.filter(pl.col(research_experience) == 3).select(pl_components)
experience_4 = by_area_per_research_experience.filter(pl.col(research_experience) == 4).select(pl_components)

anova = stats.f_oneway(experience_1, experience_2, experience_3, experience_4)
anova_research_experience = pl.DataFrame({
    "component": components,
    "f-value": anova[0],
    "p-value": anova[1],
}).write_csv("figure/5/라_5_연구경력_조사_대상별_차이_검정_anova.csv", include_bom=True)

In [53]:
# 
# 6) 연구분야
# 

research_field = "Q47"
by_area_per_research_field = group_per_personal(research_field)

count_per_research_field = compute_group_personality_by_callback(by_area_per_research_field, research_field, pl.count).rename({"column_0": "물리", "column_1": "화학", "column_2": "생물", "column_3": "지질", "column_4": "공학", "column_5": "정책", "column_6": "기타"})
mean_per_research_field = compute_group_personality_by_callback(by_area_per_research_field, research_field, pl.mean).rename({"column_0": "물리평균", "column_1": "화학평균", "column_2": "생물평균", "column_3": "지질평균", "column_4": "공학평균", "column_5": "정책평균", "column_6": "기타평균"})
std_per_research_field = compute_group_personality_by_callback(by_area_per_research_field, research_field, pl.std).rename({"column_0": "물리표준편차", "column_1": "화학표준편차", "column_2": "생물표준편차", "column_3": "지질표준편차", "column_4": "공학표준편차", "column_5": "정책표준편차", "column_6": "기타표준편차"})

all = count_per_research_field.join(
    mean_per_research_field, on="column", how="inner",
).join(
    std_per_research_field, on="column", how="inner"
)

all.write_csv("figure/5/라_6_연구분야_조사_대상별_차이_검정_통계량.csv", include_bom=True)

field_1 = by_area_per_research_field.filter(pl.col(research_field) == 1).select(pl_components)
field_2 = by_area_per_research_field.filter(pl.col(research_field) == 2).select(pl_components)
field_3 = by_area_per_research_field.filter(pl.col(research_field) == 3).select(pl_components)
field_4 = by_area_per_research_field.filter(pl.col(research_field) == 4).select(pl_components)
field_5 = by_area_per_research_field.filter(pl.col(research_field) == 5).select(pl_components)
field_6 = by_area_per_research_field.filter(pl.col(research_field) == 6).select(pl_components)
field_7 = by_area_per_research_field.filter(pl.col(research_field) == 7).select(pl_components)

anova = stats.f_oneway(field_1, field_2, field_3, field_4, field_5, field_6, field_7)
anova_research_field = pl.DataFrame({
    "component": components,
    "f-value": anova[0],
    "p-value": anova[1],
}).write_csv("figure/5/라_6_연구분야_조사_대상별_차이_검정_anova.csv", include_bom=True)

In [54]:
# 
# elastic net
# 
from attributes import *
import polars_ols as pl_components
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# 연구윤리 인식 영역, 연구윤리 교육 영역, 기관에 대한 인식, 인구통계학적 배경 변인
total = pl.read_csv("data/1_total.csv")


target = total.join(by_area, on="ID", how="left").to_pandas()
features = total.drop('ID').columns

train, test = train_test_split(target, test_size=0.3, random_state=42)

kf = KFold(n_splits=100, shuffle=True, random_state=42)
for i, (train_index, test_index) in enumerate(kf.split(train)):
    train_cv = target.iloc[train_index]
    test_cv = target.iloc[test_index]

    train_pl = pl.DataFrame(train_cv)
    test_pl = pl.DataFrame(test_cv)

    print(train_pl.shape, test_pl.shape)

    expr = pl.col("존중").least_squares.elastic_net(
        *features,
        mode="statistics",
        l1_ratio=0.5,
        alpha=0.1,
        max_iter=1000,
        tol=0.0001,
    )

    some = train_pl.select(expr).unnest('statistics')

    to_select = pl.DataFrame({
        "features": features,
        "coefficients": some['coefficients'].to_list()[0],
        "p_values": some['p_values'].to_list()[0],
        "t_values": some['t_values'].to_list()[0],
    })

    selected = to_select.filter(
        ((pl.col("coefficients") > 0) & (pl.col("p_values") < 0.05)).alias("selected")
    )
    print(selected)

(114, 124) (2, 124)
shape: (6, 4)
┌──────────┬──────────────┬──────────┬──────────┐
│ features ┆ coefficients ┆ p_values ┆ t_values │
│ ---      ┆ ---          ┆ ---      ┆ ---      │
│ str      ┆ f64          ┆ f64      ┆ f64      │
╞══════════╪══════════════╪══════════╪══════════╡
│ Q1       ┆ 0.075605     ┆ 0.011383 ┆ 2.836254 │
│ Q2       ┆ 0.148033     ┆ 0.000734 ┆ 4.106235 │
│ Q3_5     ┆ 0.024957     ┆ 0.030865 ┆ 2.353495 │
│ Q40_8    ┆ 0.02513      ┆ 0.006571 ┆ 3.094405 │
│ Q40_25   ┆ 0.076665     ┆ 0.003951 ┆ 3.330537 │
│ Q47      ┆ 0.01168      ┆ 0.002614 ┆ 3.521166 │
└──────────┴──────────────┴──────────┴──────────┘
(114, 124) (2, 124)
shape: (3, 4)
┌──────────┬──────────────┬──────────┬──────────┐
│ features ┆ coefficients ┆ p_values ┆ t_values │
│ ---      ┆ ---          ┆ ---      ┆ ---      │
│ str      ┆ f64          ┆ f64      ┆ f64      │
╞══════════╪══════════════╪══════════╪══════════╡
│ Q1       ┆ 0.066295     ┆ 0.008195 ┆ 3.006662 │
│ Q2       ┆ 0.153414     ┆ 0.02